In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float 
import pymysql
engine = create_engine('mysql+pymysql://root:Sp@ceBall5@localhost:3306/lahman2016')
connection = engine.connect()

/Users/kristopherking/anaconda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


In [23]:
# SELECTING ALL PLAYERS NOMINATED OR INDUCTED IN THE MLB HALL OF FAME
hallOF = pd.read_sql("SELECT * from `HallOfFame` WHERE Category='Player' and votedBy='BBWAA'", connection)
# CREATING A LIST OF THE HOF PLAYER IDs
hallOF_players = hallOF['playerID'].unique()
print("Number of HOF Players: " + str(len(hallOF_players)))

Number of HOF Players: 1130


In [24]:
# CREATE PICTHER ID LIST FROM ALL PITCHERS IN MLB
pitcher_id = pd.read_sql("SELECT Pos, playerID from `Fielding` where Pos='P'", connection)
pitcher_id_list = pitcher_id['playerID'].unique()
print("Number of MLB Pitchers: " + str(len(pitcher_id_list)))

Number of MLB Pitchers: 9302


In [25]:
# STORING HOF FIELDERS AND PITCHERS PLAYER IDs IN SEPERATE LISTS
fielders_HOF = []   # Excludes pitchers

pitchers_HOF = []  # Store pitchers

for h in hallOF_players:
    if h not in pitcher_id_list:
        if h not in fielders_HOF:
            fielders_HOF.append(h)
    else:
        if h not in pitchers_HOF:
            pitchers_HOF.append(h)
print("Number of HOF Fielders: " + str(len(fielders_HOF)))            
print("Number of HOF Pitchers: " + str(len(pitchers_HOF)))

Number of HOF Fielders: 681
Number of HOF Pitchers: 449


In [21]:
# COLLECT ALL UNIQUE PLAYER IDs FROM BATTING TABLE
batters = pd.read_sql("SELECT playerID from `Batting`", connection)
batter_list = batters['playerID'].unique().tolist()
print("Number of Batters (including pitchers): " + str(len(batter_list)))
# REMOVE PLAYER IDs THAT ARE IDENTIFIED AS BEING PITCHERS
for b in batter_list:
    if b in pitcher_id_list:
        batter_list.remove(b)
print("Number of Batters (without pitchers): " + str(len(batter_list)))

Number of Batters (including pitchers): 18915
Number of Batters (without pitchers): 12716


In [5]:
# STORE ALL PITCHERS (NON-HOF) THAT HAVE 10 OR MORE SEASONS IN TO A LIST BY PLAYER ID
pitchers_NonHOF = []
for p in pitcher_id_list:
    if p not in pitchers_HOF:
        pitchers_NonHOF.append(p)
print("Number of Non-HOF Pitchers: " + str(len(pitchers_NonHOF)))

Number of Non-HOF Pitchers: 8853


In [6]:
# CREATE DF FROM ALL PITCHERS
pitchers = pd.read_sql("SELECT * from `Pitching` where playerID in %s" % str(tuple(pitcher_id_list)), connection)
pitch = pd.DataFrame(pitchers)
# REPLACE BLANK VALUES
pitch.replace('', 0, inplace=True)
pitch.replace(0, 1, inplace=True)
pitch.head()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NA,1,2,3,3,2,...,1,1,1,1,1,1,42,1,1,1
1,brainas01,1871,1,WS3,NA,12,15,30,30,30,...,1,1,1,1,1,1,292,1,1,1
2,fergubo01,1871,1,NY2,NA,1,1,1,1,1,...,1,1,1,1,1,1,9,1,1,1
3,fishech01,1871,1,RC1,NA,4,16,24,24,22,...,1,1,1,1,1,1,257,1,1,1
4,fleetfr01,1871,1,NY2,NA,1,1,1,1,1,...,1,1,1,1,1,1,21,1,1,1


In [7]:
pitching_df = pd.DataFrame()
x = 0
#LOOP THROUGH THE PITCHERS LIST
for p in pitcher_id_list[:-1]:
    a = pd.read_sql("SELECT * from `Pitching` where playerID='%s'" % p, connection)
    # SELECT PLAYERS WITH 10 OR MORE SEASONS
    if len(a) >= 10:
        print('Running data on %s: record %s of %s' %(p, x, len(pitcher_id_list)-1))
        name_pull = pd.read_sql("SELECT nameLast, NameGiven from `Master` where playerID = '%s'"% p, connection)
        lastName = name_pull['nameLast'][0]
        firstName = name_pull['NameGiven'][0]
        pit = pitch[(pitch.playerID==p)]
        pit[['W','L','G','GS','CG','SHO','SV','IPouts','H','ER','HR','BB','SO','BAOpp','ERA','IBB','WP','HBP',
      'BK','BFP','GF','R','SH','SF','GIDP']].astype(str).astype(float)
        z = pit['playerID'].count()
        win = pit['W'].sum()
        loss = pit['L'].sum()
        games = pit['G'].sum()
        games_started = pit['GS'].sum()
        complete = pit['CG'].sum()
        shutout = pit['SHO'].sum()
        save = pit['SV'].sum()
        ipOut = pit['IPouts'].sum()
        print(len(pit))
        era = pit['ERA'].sum()/len(pit)
        runs = pit['ER'].sum()
        hits = pit['H'].sum()
        hr = pit['HR'].sum()
        OppBA = pit['BAOpp'].astype(str).astype(float).sum()/z
        strikes = pit['SO'].sum()
        walks = pit['BB'].sum()
        if p in pitchers_HOF:
            hof = 'Y'
        else:
            hof = 'N'
        #APPEND STATS TO A DF
        pitching_df = pitching_df.append({
            'player_id': p,
            'firstName': firstName,
            'lastName': lastName, 
            'games': games,
            'wins': win,
            'losses': loss,
            'started': games_started,
            'completed': complete,
            'shutouts': shutout,
            'saves': save,
            'outs_pitched': ipOut,
            'ERA': era,
            'runs_earned': runs,
            'hits': hits,
            'homeruns': hr,
            'Opposing_BA': OppBA,
            'strikes': strikes,
            'walks': walks,
            'HOF': hof
        }, ignore_index = True)
        x+=1
    else:
        print('Record for %s skipped' % p)
        next

Record for bechtge01 skipped
Record for brainas01 skipped
Record for fergubo01 skipped
Record for fishech01 skipped
Record for fleetfr01 skipped
Record for flowedi01 skipped
Record for mackde01 skipped
Running data on mathebo01: record 0 of 9301
16
Record for mcbridi01 skipped
Record for mcmuljo01 skipped
Record for meyerle01 skipped
Record for paborch01 skipped
Record for pinkhed01 skipped
Record for prattal01 skipped
Record for spaldal01 skipped
Record for stearbi01 skipped
Record for woltery01 skipped
Record for wrighha01 skipped
Record for zettlge01 skipped
Record for bentlcy01 skipped
Record for brittji01 skipped
Record for buttefr01 skipped
Record for cummica01 skipped
Record for malonma01 skipped
Record for martiph01 skipped
Record for mcderjo01 skipped
Record for orour01 skipped
Record for campbhu01 skipped
Record for forceda01 skipped
Record for frencbi01 skipped
Record for fulmech01 skipped
Record for greasjo01 skipped
Record for lovetle01 skipped
Record for mcdoo01 skipped
R

Record for kimbesa01 skipped
Record for kirbyjo01 skipped
Record for knighjo01 skipped
Record for kreegfr01 skipped
Record for lauerch01 skipped
Record for leeto01 skipped
Record for lockwmi01 skipped
Record for lynchto02 skipped
Record for macarma01 skipped
Record for mattecv01 skipped
Running data on maulal01: record 18 of 9301
16
Record for mccarto01 skipped
Record for mccaual01 skipped
Record for mccorje01 skipped
Record for mcelrji01 skipped
Record for mcfarch01 skipped
Record for mckeola01 skipped
Record for mclauba01 skipped
Record for mclauji01 skipped
Record for mcsortr01 skipped
Record for meegape01 skipped
Record for meinkfr01 skipped
Record for millecy01 skipped
Record for moffesa01 skipped
Record for moriage01 skipped
Record for morrie01 skipped
Record for morried01 skipped
Record for mortoch01 skipped
Record for mortosp01 skipped
Record for murphco01 skipped
Record for murphjo01 skipped
Record for nelsobi01 skipped
Record for oberbhe01 skipped
Record for obriebi01 skipped

Record for doefr01 skipped
Record for dolanjo01 skipped
Record for dotyba01 skipped
Record for dowseto01 skipped
Record for duzenbi01 skipped
Record for eiteled01 skipped
Running data on esperdu01: record 36 of 9301
13
Record for fieldji01 skipped
Record for fitzgjo01 skipped
Record for fordto01 skipped
Record for francos01 skipped
Record for germale01 skipped
Record for gibsoro01 skipped
Record for gleasbi02 skipped
Record for goodahe01 skipped
Record for graych01 skipped
Record for greened01 skipped
Record for grimjo01 skipped
Record for gumbebi01 skipped
Record for hartbi02 skipped
Record for haynefr01 skipped
Record for heardch01 skipped
Record for helmbho01 skipped
Running data on hemmige01: record 37 of 9301
11
Record for hustebi01 skipped
Record for joneshe02 skipped
Record for jonesmi01 skipped
Record for keefejo01 skipped
Record for keffefr01 skipped
Record for knausfr01 skipped
Record for lacke01 skipped
Record for lawsoal01 skipped
Record for lewis01 skipped
Record for linco

Record for dammabi01 skipped
Record for denzero01 skipped
Record for dowlipe01 skipped
Record for dunklda01 skipped
Record for dunnja01 skipped
Record for evansro01 skipped
Record for fifieja01 skipped
Record for grimejo01 skipped
Record for hickmch01 skipped
Record for johnsyo01 skipped
Record for jonesji01 skipped
Record for lippto01 skipped
Record for mageebi01 skipped
Record for mahonmi01 skipped
Record for millebe01 skipped
Record for pappajo01 skipped
Running data on powelja01: record 54 of 9301
17
Running data on sparktu01: record 55 of 9301
13
Record for stanljo02 skipped
Running data on sudhowi01: record 56 of 9301
11
Record for swaimcy01 skipped
Running data on wadderu01: record 57 of 9301
14
Running data on altroni01: record 58 of 9301
18
Record for batesfr01 skipped
Record for callajo02 skipped
Record for carribi01 skipped
Record for connbe01 skipped
Running data on dinnebi01: record 59 of 9301
13
Running data on donovbi01: record 60 of 9301
18
Running data on dugglbi01: re

Record for sandewa01 skipped
Record for scanldo01 skipped
Record for schmihe01 skipped
Record for stovaje01 skipped
Record for tayloed01 skipped
Record for thatcgr01 skipped
Record for thompgu01 skipped
Record for veilbu01 skipped
Record for washbli01 skipped
Record for weimeja01 skipped
Record for wiggsji01 skipped
Record for wilheka01 skipped
Record for wolfeba01 skipped
Record for applefr01 skipped
Record for barryto01 skipped
Record for barthjo01 skipped
Record for bowerfr01 skipped
Record for brackjo01 skipped
Record for brucelo01 skipped
Record for caldwra01 skipped
Running data on camniho01: record 89 of 9301
12
Record for clarkwa01 skipped
Record for corrifr01 skipped
Record for delahji01 skipped
Record for doughto01 skipped
Record for durhabu01 skipped
Record for elliocl01 skipped
Record for ferrycy01 skipped
Record for fisheto01 skipped
Record for grother01 skipped
Record for hickeja01 skipped
Record for jacobbe01 skipped
Record for jaegech01 skipped
Record for koukajo01 skip

...

Record for motteja01 skipped
Record for niemaje01 skipped
Running data on niesejo01: record 1477 of 9301
10
Running data on odayda01: record 1478 of 9301
10
Record for outmajo01 skipped
Record for palmema01 skipped
Record for parismi01 skipped
Record for parnebo01 skipped
Record for parrja01 skipped
Record for pattesc01 skipped
Record for penato02 skipped
Record for perezch01 skipped
Running data on priceda01: record 1479 of 9301
11
Record for purceda01 skipped
Record for ramirra03 skipped
Record for regisst01 skipped
Record for reinech01 skipped
Record for reynogr01 skipped
Running data on richacl01: record 1480 of 9301
10
Record for richmsc01 skipped
Record for roberda08 skipped
Record for roenijo01 skipped
Record for romose01 skipped
Record for rosaca01 skipped
Record for rosalle01 skipped
Record for rundlri01 skipped
Record for russead01 skipped
Record for salasma02 skipped
Running data on samarje01: record 1481 of 9301
10
Record for sanchhu01 skipped
Record for scherma01 skipped
R

Record for sanabal01 skipped
Record for santose01 skipped
Record for sborzja01 skipped
Record for schlibr01 skipped
Record for sinkbbr01 skipped
Record for slamaan01 skipped
Record for smithjo06 skipped
Record for stangda01 skipped
Record for storedr01 skipped
Record for strasst01 skipped
Record for takahhi01 skipped
Record for texeika01 skipped
Record for tomlijo01 skipped
Record for valdece01 skipped
Record for valdera02 skipped
Record for varvaan01 skipped
Record for ventejo01 skipped
Record for villahe02 skipped
Record for waldejo01 skipped
Record for weinhro01 skipped
Record for woodbl01 skipped
Record for woodtr01 skipped
Record for worleva01 skipped
Record for abreuju01 skipped
Record for adcocna01 skipped
Record for albural01 skipped
Record for alvarhe01 skipped
Record for axelrdy01 skipped
Record for bassan01 skipped
Record for beatope01 skipped
Record for beavabl01 skipped
Record for beckch01 skipped
Record for belowdu01 skipped
Record for betande01 skipped
Record for billibr

Record for alberan01 skipped
Record for alvarjo02 skipped
Record for amesst01 skipped
Record for belfimi01 skipped
Record for bettich01 skipped
Record for blackvi01 skipped
Record for blazemi01 skipped
Record for boshebu01 skipped
Record for brasiry01 skipped
Record for brewech01 skipped
Record for brittdr01 skipped
Record for burgohi01 skipped
Record for butleke01 skipped
Record for cabrace01 skipped
Record for caminar01 skipped
Record for carroja01 skipped
Record for castrsi01 skipped
Record for chapmke02 skipped
Record for chrisni01 skipped
Record for cisnejo01 skipped
Record for claibpr01 skipped
Record for clarkza01 skipped
Record for clemepa02 skipped
Record for colege01 skipped
Record for colomal01 skipped
Record for cosarja01 skipped
Record for cumptbr01 skipped
Record for daviser02 skipped
Record for delareu01 skipped
Record for delatjo01 skipped
Record for deleojo02 skipped
Record for dominjo01 skipped
Record for dunnija01 skipped
Record for dwyerch01 skipped
Record for elmor

Record for anderco01 skipped
Record for andrima01 skipped
Record for araujel01 skipped
Record for armstsh01 skipped
Record for arojo01 skipped
Record for asheral01 skipped
Record for banuema01 skipped
Record for barraky01 skipped
Record for barriyh01 skipped
Record for beckch02 skipped
Record for bellaan01 skipped
Record for boydma01 skipped
Record for brachsi01 skipped
Record for bradlar01 skipped
Record for brighja01 skipped
Record for broadmi01 skipped
Record for burawda01 skipped
Record for burgoen02 skipped
Record for castran01 skipped
Record for castrmi01 skipped
Record for coleaj01 skipped
Record for conlead01 skipped
Record for cooneti01 skipped
Record for copelsc01 skipped
Record for cornejo01 skipped
Record for cothaca01 skipped
Record for cravyty01 skipped
Record for cunnibr02 skipped
Record for davieza01 skipped
Record for davisik02 skipped
Record for deckeja01 skipped
Record for delosab01 skipped
Record for denorch01 skipped
Record for depaujo02 skipped
Record for drakeol0

Record for perdolu02 skipped
Record for phegljo01 skipped
Record for plutkad01 skipped
Record for poundbr01 skipped
Record for purkema01 skipped
Record for ramosed02 skipped
Record for reedco01 skipped
Record for reyesal02 skipped
Record for rodgebr01 skipped
Record for rodrijo04 skipped
Record for rogerta01 skipped
Record for sampsad01 skipped
Record for sardilu01 skipped
Record for saupowa01 skipped
Record for schuspa01 skipped
Record for scottro02 skipped
Record for shiplbr01 skipped
Record for shuckja01 skipped
Record for smithbl01 skipped
Record for smithja06 skipped
Record for smokejo01 skipped
Record for snellbl01 skipped
Record for somsela01 skipped
Record for stephro01 skipped
Record for stewabr01 skipped
Record for strahma01 skipped
Record for stratch01 skipped
Record for stripro01 skipped
Record for stumpda01 skipped
Record for sturdty01 skipped
Record for suareal01 skipped
Record for suterbr01 skipped
Record for taillja01 skipped
Record for tejadru01 skipped
Record for thom

In [9]:
# CONVERT FINAL PITCHING DF TO CSV
# INCLUDES ONLY PITCHERS WITH 10 OR MORE SEASONS
# HOF COLUMN DENOTES WHETHER NOMINATED(OR INDUCTED) IN THE HALL OF FAME WITH A 'Y' OR 'N'
pitching_df.to_csv('pitchersW10YRS.csv')

In [8]:
pitching_df.shape

(1491, 19)

In [10]:
pitching_df.head()

,ERA,HOF,Opposing_BA,completed,firstName,games,hits,homeruns,lastName,losses,outs_pitched,player_id,runs_earned,saves,shutouts,started,strikes,walks,wins
0,3.250625,N,0.725000,525.0,Robert T.,578.0,5591.0,71.0,Mathews,249.0,14869.0,mathebo01,1591.0,17.0,25.0,568.0,1366.0,533.0,297.0
1,2.956364,N,0.528182,387.0,Thomas Henry,417.0,3767.0,52.0,Bond,163.0,10886.0,bondto01,932.0,11.0,46.0,408.0,879.0,198.0,237.0
2,3.001765,N,0.961765,646.0,James Francis,705.0,6405.0,124.0,Galvin,310.0,18010.0,galvipu01,1910.0,17.0,60.0,689.0,1806.0,745.0,364.0
3,2.855000,Y,0.626000,394.0,William Henry,403.0,3440.0,67.0,White,166.0,10628.0,whitewi01,896.0,10.0,38.0,401.0,1041.0,496.0,230.0
4,2.416667,N,0.303333,466.0,James,492.0,4092.0,84.0,McCormick,214.0,12827.0,mccorji01,1155.0,12.0,35.0,485.0,1704.0,749.0,265.0


In [13]:
hof_pitchersDF = pitching_df[(pitching_df.HOF == 'Y')]
print(hof_pitchersDF.shape)
hof_pitchersDF.head()

(372, 19)


,ERA,HOF,Opposing_BA,completed,firstName,games,hits,homeruns,lastName,losses,outs_pitched,player_id,runs_earned,saves,shutouts,started,strikes,walks,wins
3,2.855000,Y,0.626000,394.0,William Henry,403.0,3440.0,67.0,White,166.0,10628.0,whitewi01,896.0,10.0,38.0,401.0,1041.0,496.0,230.0
33,3.721000,Y,0.252000,392.0,Amos Wilson,462.0,3384.0,76.0,Rusie,174.0,11309.0,rusieam01,1287.0,10.0,31.0,427.0,1934.0,1704.0,246.0
38,3.470625,Y,0.256250,531.0,Charles Augustus,620.0,4912.0,157.0,Nichols,208.0,15169.0,nichoki01,1660.0,25.0,50.0,561.0,1868.0,1268.0,362.0
39,2.713913,Y,0.248696,749.0,Denton True,906.0,7092.0,139.0,Young,316.0,22064.0,youngcy01,2147.0,30.0,80.0,815.0,2803.0,1217.0,511.0
40,4.060000,Y,0.334545,300.0,Theodore J.,379.0,3091.0,79.0,Breitenstein,171.0,8893.0,breitte01,1330.0,11.0,14.0,341.0,889.0,1203.0,161.0


In [ ]:
len(batter_list)

In [32]:
fielders_all = pd.read_sql("SELECT * from `Batting` where playerID in %s" % str(tuple(batter_list)), connection)
fieldersDF = pd.DataFrame(fielders_all)
fieldersDF.replace('', 0.01, inplace=True)
fieldersDF.replace(0, 0.01, inplace=True)
fieldersDF.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NA,1,4.0,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
1,addybo01,1871,1,RC1,NA,25,118.0,30.00,32.00,6.00,...,13.00,8.00,1.00,4.00,0.01,0.01,0.01,0.01,0.01,0.01
2,allisar01,1871,1,CL1,NA,29,137.0,28.00,40.00,4.00,...,19.00,3.00,1.00,2.00,5.00,0.01,0.01,0.01,0.01,0.01
3,ansonca01,1871,1,RC1,NA,25,120.0,29.00,39.00,11.00,...,16.00,6.00,2.00,2.00,1.00,0.01,0.01,0.01,0.01,0.01
4,armstbo01,1871,1,FW1,NA,12,49.0,9.00,11.00,2.00,...,5.00,0.01,1.00,0.01,1.00,0.01,0.01,0.01,0.01,0.01


In [34]:
batting_df = pd.DataFrame()

x = 0
for t in batter_list[:-1]:
    a = pd.read_sql("SELECT * from `Batting` where playerID='%s'"% t, connection)
    if len(a) >= 10:
        print('Running data on %s: record %s of %s' %(t,x, len(batter_list)-1))
        name_pull = pd.read_sql("SELECT nameLast, NameGiven from `Master` where playerID = '%s'"% t, connection)
        lastName = name_pull['nameLast'][0]
        firstName = name_pull['NameGiven'][0]
        p = fieldersDF[(fieldersDF.playerID==t)]
        games = p['G'].sum()
        ab = p['AB'].sum()
        runs = p['R'].sum()
        hits = p['H'].sum()
        _2b = p['2B'].sum()
        _3b = p['3B'].sum()
        hr = p['HR'].sum()
#         extra = _2b + _3b + hr
#         _1b = ab - extra
        bb = p['BB'].sum()
        p['IBB'] = p['IBB'].astype(str).astype(float)
        ibb = p['IBB'].sum()
        p['HBP']=p['HBP'].astype(str).astype(float)
        hbp = p['HBP'].sum()
        p['SF'] = p['SF'].astype(str).astype(float)
        sf = p['SF'].sum()
        _BA = hits / ab
        obp = (hits + bb + hbp) / (ab + bb + hbp + sf)
        slg = ((hits) + (_2b) + (_3b * 2) + (hr*3))/ab
        ops = obp + slg
        sb = p['SB'].sum()
        if t in fielders_HOF:
            hof = 'Y'
        else:
            hof = 'N'
        batting_df = batting_df.append({
            'player_id': t,
            'firstName': firstName,
            'lastName': lastName,
            'games': games,
            'runs': runs,
            'hits': hits,
#             '1B': _1b,
            '2B': _2b,
            '3B': _3b,
            'HR': hr,
            'extra_base_hits': extra,
            'BB': bb,
            'IBB':ibb,
            'HBP':hbp,
            'SF': sf,
            'SB':sb,
            'BA': _BA,
            'OBP':obp,
            'SLG':slg,
            'OPS':ops,
            'HOF':hof
        }, ignore_index=True)

        x+=1
    else:
        print('Record for %s skipped' % t)
        next

Record for abercda01 skipped
Record for addybo01 skipped
Record for allisar01 skipped
Running data on ansonca01: record 0 of 12715


/Users/kristopherking/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kristopherking/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kristopherking/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

Record for armstbo01 skipped
Record for barkeal01 skipped
Record for barrebi01 skipped
Record for barrofr01 skipped
Record for bassjo01 skipped
Record for bealsto01 skipped
Record for beaveed01 skipped
Record for bellast01 skipped
Record for berkena01 skipped
Record for berryto01 skipped
Record for berthha01 skipped
Record for biermch01 skipped
Record for birdge01 skipped
Record for birdsda01 skipped
Record for brannmi01 skipped
Record for burrohe01 skipped
Record for careyto01 skipped
Record for carleji01 skipped
Record for conefr01 skipped
Record for connone01 skipped
Record for cravebi01 skipped
Running data on cuthbne01: record 1 of 12715
Record for deaneha01 skipped
Record for donnepe01 skipped
Record for duffyed01 skipped
Running data on eggleda01: record 2 of 12715
Record for ewellge01 skipped
Record for fishech01 skipped
Record for fislewe01 skipped
Record for flowedi01 skipped
Record for flynncl01 skipped
Record for foleyto01 skipped
Record for foranji01 skipped
Running data o

Record for treacpe01 skipped
Record for valenbo01 skipped
Record for wardji01 skipped
Record for whitnfr01 skipped
Record for buncejo01 skipped
Record for edench01 skipped
Record for gleasja01 skipped
Record for haldejo01 skipped
Record for leele01 skipped
Record for littlha01 skipped
Record for loftuto01 skipped
Record for mckenpa01 skipped
Record for millege01 skipped
Record for newelte01 skipped
Record for pikeja01 skipped
Record for roweda01 skipped
Record for smithha01 skipped
Record for sullich01 skipped
Record for tayloli01 skipped
Running data on bennech01: record 23 of 12715
Record for blissfr01 skipped
Record for creamge01 skipped
Running data on dalryab01: record 24 of 12715
Running data on dickebu01: record 25 of 12715
Record for goodmja01 skipped
Record for healeto01 skipped
Record for jennial01 skipped
Record for mcclebi01 skipped
Record for mckelru01 skipped
Record for powerph01 skipped
Record for sullibi01 skipped
Record for traffbi01 skipped
Running data on wheelha01: 

Record for brennji01 skipped
Record for briggch01 skipped
Record for brownji01 skipped
Record for bukerha01 skipped
Record for bullasi01 skipped
Record for burcher01 skipped
Record for burnspa01 skipped
Record for butlebi01 skipped
Record for butleki01 skipped
Record for callaed01 skipped
Record for callapa01 skipped
Record for carroch01 skipped
Record for carrofr01 skipped
Record for carropa01 skipped
Record for carrosc01 skipped
Record for caseyda01 skipped
Record for caseyde01 skipped
Record for chattji01 skipped
Running data on clemeja01: record 61 of 12715
Record for cleveel01 skipped
Record for colgaed01 skipped
Record for collich01 skipped
Record for connojo01 skipped
Record for conwabi01 skipped
Record for cookpa01 skipped
Record for corcomi01 skipped
Record for corriph01 skipped
Record for coxfr01 skipped
Record for creegma01 skipped
Record for cronida01 skipped
Record for crosscl01 skipped
Record for crowljo01 skipped
Record for cuffjo01 skipped
Record for cullejo01 skipped
R

...

Record for bartobr01 skipped
Record for bernaro01 skipped
Record for bernido01 skipped
Record for bixlebr01 skipped
Record for blancgr01 skipped
Record for bococbr01 skipped
Record for boggsbr01 skipped
Record for bonined01 skipped
Record for bourgja01 skipped
Record for bowerce01 skipped
Record for bowkejo01 skipped
Running data on brignre01: record 2660 of 12715
Running data on bruceja01: record 2661 of 12715
Record for burriem01 skipped
Record for carlilu01 skipped
Record for carpean01 skipped
Record for cartech01 skipped
Record for castiwi01 skipped
Record for cervefr01 skipped
Record for cervemi01 skipped
Record for chrisju01 skipped
Record for conrabr01 skipped
Record for crabbca01 skipped
Record for cruzlu01 skipped
Record for cunniaa01 skipped
Record for dantoja01 skipped
Record for delgaje01 skipped
Record for denketr01 skipped
Record for dewitbl01 skipped
Record for diazro01 skipped
Record for dickech01 skipped
Record for dillati01 skipped
Record for durange01 skipped
Record 

Record for whislwe01 skipped
Record for wietema01 skipped
Running data on younger03: record 2681 of 12715
Record for zimmejo02 skipped
Record for alonsyo01 skipped
Record for alvarpe01 skipped
Record for anderbr05 skipped
Record for anderla03 skipped
Record for arrieja01 skipped
Record for barneda01 skipped
Record for baxtemi01 skipped
Record for belljo01 skipped
Record for boescbr01 skipped
Record for boscajc01 skipped
Record for bourjpe01 skipped
Record for browndo01 skipped
Record for brownjo05 skipped
Record for burneal01 skipped
Record for cainlo01 skipped
Record for cartech02 skipped
Record for cassebo01 skipped
Record for castiwe01 skipped
Record for castrja01 skipped
Record for castrst01 skipped
Record for chapmar01 skipped
Record for ciriape01 skipped
Record for coellro01 skipped
Record for congeha01 skipped
Record for cousisc01 skipped
Record for craigal01 skipped
Record for curtico01 skipped
Record for davisbr03 skipped
Record for dedunsa01 skipped
Record for delroen01 skipp

Record for adamsma01 skipped
Record for aokino01 skipped
Record for aumonph01 skipped
Record for averyxa01 skipped
Record for barnebr02 skipped
Record for bauertr01 skipped
Record for berryqu01 skipped
Record for biancje01 skipped
Record for brantro01 skipped
Record for brummty01 skipped
Record for burnsco01 skipped
Record for cabreed01 skipped
Record for calhoko01 skipped
Record for cardead01 skipped
Record for carsoro01 skipped
Record for casteal01 skipped
Record for cespeyo01 skipped
Record for chenwe02 skipped
Record for cloydty01 skipped
Record for costami01 skipped
Record for crosbca01 skipped
Record for culbech01 skipped
Record for danksjo02 skipped
Record for devrico01 skipped
Record for diazju02 skipped
Record for doolise01 skipped
Record for doziebr01 skipped
Record for eatonad02 skipped
Record for elmorja01 skipped
Record for exposlu01 skipped
Record for faluir01 skipped
Record for fickch01 skipped
Record for figuepe01 skipped
Record for fontwi01 skipped
Record for friedch01

Record for altheaa01 skipped
Record for alvarrj01 skipped
Record for annade01 skipped
Record for arrueer01 skipped
Record for baezja01 skipped
Record for barnema01 skipped
Record for barnhtu01 skipped
Record for bassich01 skipped
Record for beeleda01 skipped
Record for belnovi01 skipped
Record for bettsmo01 skipped
Record for bolsimi01 skipped
Record for bourju01 skipped
Record for brentbr01 skipped
Record for brownbr02 skipped
Record for brownga02 skipped
Record for buchaja01 skipped
Record for burnsbi02 skipped
Record for butleda01 skipped
Record for campber01 skipped
Record for carrosc02 skipped
Record for casalcu01 skipped
Record for castiru01 skipped
Record for cecchga01 skipped
Record for clarkma02 skipped
Record for collity01 skipped
Record for colonch01 skipped
Record for corcida01 skipped
Record for couloda01 skipped
Record for croncj01 skipped
Record for degroja01 skipped
Record for desclan01 skipped
Record for diazja01 skipped
Record for dominch01 skipped
Record for duffyma0

Record for sweenda02 skipped
Record for swihabl01 skipped
Record for tartatr01 skipped
Record for thomptr01 skipped
Record for tomasya01 skipped
Record for tomlike01 skipped
Record for torrero01 skipped
Record for travide01 skipped
Record for tuckepr01 skipped
Record for turnetr01 skipped
Record for urshegi01 skipped
Record for velasvi01 skipped
Record for verrelo01 skipped
Record for waldrky02 skipped
Record for willima07 skipped
Record for willima08 skipped
Record for winklda01 skipped
Record for wojcias01 skipped
Record for zychto01 skipped
Record for alcanra01 skipped
Record for alfarjo01 skipped
Record for almoral01 skipped
Record for anderti01 skipped
Record for antoldu01 skipped
Record for arciaor01 skipped
Record for asuajca01 skipped
Record for austity01 skipped
Record for barneda02 skipped
Record for barneto01 skipped
Record for baumabu01 skipped
Record for belljo02 skipped
Record for beninan01 skipped
Record for beresja01 skipped
Record for biagijo01 skipped
Record for blair

In [46]:
print(batting_df.shape)
batting_df.head()

(2683, 20)


,2B,3B,BA,BB,HBP,HOF,HR,IBB,OBP,OPS,SB,SF,SLG,extra_base_hits,firstName,games,hits,lastName,player_id,runs
0,581.00,142.00,0.332587,983.00,32.16,N,97.10,0.27,0.392578,0.837679,276.11,0.27,0.445101,0.0,Adrian Constantine,2523.0,3418.00,Anson,ansonca01,1996.00
1,74.00,17.04,0.254141,65.00,0.10,N,8.06,0.10,0.276421,0.593155,69.05,0.10,0.316735,0.0,Edgar Edward,452.0,537.00,Cuthbert,cuthbne01,453.00
2,85.01,31.05,0.274156,48.00,0.12,N,0.12,0.12,0.287606,0.619684,47.07,0.12,0.332078,0.0,David Daniel,576.0,698.00,Eggler,eggleda01,491.01
3,70.02,30.02,0.260660,42.02,0.10,N,8.08,0.12,0.273193,0.597090,14.09,0.12,0.323898,0.0,Charles John,583.0,636.01,Fulmer,fulmech01,360.01
4,56.02,13.04,0.275695,26.02,0.12,N,0.12,0.12,0.283233,0.592165,65.07,0.12,0.308932,0.0,Michael Henry,547.0,684.00,McGeary,mcgeami01,484.00


In [45]:
# CONVERT FINAL BATTING DF TO CSV
# INCLUDES ONLY FIELDERS WITH 10 OR MORE SEASONS
# HOF COLUMN DENOTES WHETHER NOMINATED(OR INDUCTED) IN THE HALL OF FAME WITH A 'Y' OR 'N'
batting_df.to_csv('battersW10YRS.csv')

In [47]:
batting_df[(batting_df['BA'] > .300)]

,2B,3B,BA,BB,HBP,HOF,HR,IBB,OBP,OPS,SB,SF,SLG,extra_base_hits,firstName,games,hits,lastName,player_id,runs
0,581.00,142.00,0.332587,983.00,32.16,N,97.10,0.27,0.392578,0.837679,276.11,0.27,0.445101,0.0,Adrian Constantine,2523.0,3418.00,Anson,ansonca01,1996.00
5,119.03,53.02,0.321236,49.01,0.10,N,20.05,0.11,0.337621,0.802689,47.06,0.11,0.465068,0.0,Lipman Emanuel,425.0,637.01,Pike,pikeli01,433.01
12,465.01,151.01,0.310758,510.01,36.15,N,62.05,0.23,0.352339,0.775171,224.11,0.23,0.422831,0.0,James Henry,1999.0,2643.00,O'Rourke,orourji01,1729.00
31,262.01,94.00,0.300915,717.00,30.08,N,46.03,0.15,0.386466,0.797163,170.07,0.15,0.410696,0.0,George F.,1310.0,1612.00,Gore,gorege01,1327.00
35,441.00,233.01,0.316526,1002.00,38.07,N,138.00,0.19,0.396985,0.883003,244.06,0.19,0.486017,0.0,Roger,1997.0,2467.00,Connor,connoro01,1620.00
58,198.01,108.02,0.342052,98.03,24.03,N,37.02,0.10,0.365585,0.867294,66.05,0.10,0.501709,0.0,David L.,791.0,1125.01,Orr,orrda01,536.01
65,259.00,114.00,0.302883,484.00,100.00,N,53.00,0.10,0.379661,0.819466,129.02,0.10,0.439805,0.0,Henry E.,1184.0,1429.00,Larkin,larkihe01,925.00
74,244.00,69.02,0.310433,621.01,82.01,N,62.02,0.13,0.407436,0.850170,224.01,0.13,0.442734,0.0,Dennis Patrick Aloysius,1121.0,1333.00,Lyons,lyonsde01,932.00
75,340.02,160.00,0.330715,450.00,63.02,N,127.02,0.15,0.383555,0.888247,229.03,0.15,0.504693,0.0,Samuel Luther,1407.0,1979.00,Thompson,thompsa01,1256.00
89,272.00,158.00,0.302322,635.00,39.00,N,66.01,0.13,0.364483,0.780888,323.00,0.13,0.416405,0.0,Edwin John,1654.0,2083.00,McKean,mckeaed01,1227.00
